In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

26461


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
26456,2021-07-27,47,沖縄県,354,23201,0,233
26457,2021-07-28,47,沖縄県,347,23548,0,233
26458,2021-07-29,47,沖縄県,392,23940,2,235
26459,2021-07-30,47,沖縄県,382,24322,1,236
26460,2021-07-31,47,沖縄県,439,24761,0,236


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,12386,10885,157
43,香川県,20831,2272,32
44,高知県,21957,2087,29
45,鳥取県,17453,773,2
46,鹿児島県,25898,3968,39


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
26454,2021-07-25,47,沖縄県,209,22731,3,231,919.0
26455,2021-07-26,47,沖縄県,116,22847,2,233,1000.0
26456,2021-07-27,47,沖縄県,354,23201,0,233,1200.0
26457,2021-07-28,47,沖縄県,347,23548,0,233,1378.0
26458,2021-07-29,47,沖縄県,392,23940,2,235,1617.0
26459,2021-07-30,47,沖縄県,382,24322,1,236,1899.0
26460,2021-07-31,47,沖縄県,439,24761,0,236,2239.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
26451,2021-07-22,47,沖縄県,153,22323,0,224,738.0,-0.094675,0.442769,1.550000
26452,2021-07-23,47,沖縄県,100,22423,0,224,762.0,-0.346405,0.355187,0.315789
26453,2021-07-24,47,沖縄県,99,22522,4,228,780.0,-0.010000,0.344360,0.222222
26454,2021-07-25,47,沖縄県,209,22731,3,231,919.0,1.111111,0.522491,1.985714
26455,2021-07-26,47,沖縄県,116,22847,2,233,1000.0,-0.444976,0.530351,2.314286
26456,2021-07-27,47,沖縄県,354,23201,0,233,1200.0,2.051724,0.337740,1.298701
26457,2021-07-28,47,沖縄県,347,23548,0,233,1378.0,-0.019774,0.321001,1.053254
26458,2021-07-29,47,沖縄県,392,23940,2,235,1617.0,0.129683,0.353052,1.562092
26459,2021-07-30,47,沖縄県,382,24322,1,236,1899.0,-0.025510,0.398894,2.820000
26460,2021-07-31,47,沖縄県,439,24761,0,236,2239.0,0.149215,0.421639,3.434343


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
562,2021-07-31,北海道,284,1,1409.0,0.136000,0.152165,1.40678
1125,2021-07-31,青森県,22,1,90.0,0.466667,NaN,inf
1688,2021-07-31,岩手県,21,0,103.0,0.105263,0.133294,0.75000
2251,2021-07-31,宮城県,65,0,255.0,0.300000,0.859435,12.00000
2814,2021-07-31,秋田県,7,0,30.0,0.000000,0.521429,2.50000


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,13512,5762,114,2021-07-31,44,0,201.0,0.333333,0.250256,3.000000
1,京都府,14638,18331,249,2021-07-31,199,0,956.0,0.191617,0.362372,4.852941
2,佐賀県,23083,2747,24,2021-07-31,23,0,111.0,-0.080000,0.632324,6.666667
3,兵庫県,15764,43987,1316,2021-07-31,329,0,1600.0,0.241509,0.460220,3.633803
4,北海道,563,44424,1425,2021-07-31,284,1,1409.0,0.136000,0.152165,1.406780


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,13512,5762,114,2021-07-31,44,0,201.0,0.333333,0.250256,3.000000,1813859
1,京都府,14638,18331,249,2021-07-31,199,0,956.0,0.191617,0.362372,4.852941,2545899
2,佐賀県,23083,2747,24,2021-07-31,23,0,111.0,-0.080000,0.632324,6.666667,823810
3,兵庫県,15764,43987,1316,2021-07-31,329,0,1600.0,0.241509,0.460220,3.633803,5549568
4,北海道,563,44424,1425,2021-07-31,284,1,1409.0,0.136000,0.152165,1.406780,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,563,44424,1425,2021-07-31,284,1,1409.0,0.136000,0.152165,1.406780,5267762,84.331828,843.318282,26.747602
41,青森県,1126,2698,32,2021-07-31,22,1,90.0,0.466667,NaN,inf,1275783,21.147797,211.477971,7.054491
19,岩手県,1689,2002,47,2021-07-31,21,0,103.0,0.105263,0.133294,0.750000,1235517,16.203743,162.037430,8.336591
11,宮城県,2252,9906,91,2021-07-31,65,0,255.0,0.300000,0.859435,12.000000,2292385,43.212637,432.126366,11.123786
36,秋田県,2815,1020,20,2021-07-31,7,0,30.0,0.000000,0.521429,2.500000,985416,10.350958,103.509584,3.044400
15,山形県,3378,2141,47,2021-07-31,15,0,49.0,1.142857,NaN,14.000000,1082296,19.782019,197.820190,4.527412
35,福島県,3941,5756,162,2021-07-31,82,0,467.0,-0.057471,0.377560,4.466667,1881981,30.584793,305.847934,24.814278
38,茨城県,4504,12461,168,2021-07-31,172,1,999.0,-0.225225,0.271686,2.909091,2921436,42.653681,426.536813,34.195512
27,栃木県,5067,8324,85,2021-07-31,170,0,738.0,0.416667,0.322247,4.483871,1965516,42.350202,423.502022,37.547392
37,群馬県,5630,8792,155,2021-07-31,136,0,525.0,0.283019,0.325080,4.666667,1969439,44.642154,446.421544,26.657337


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      19632.680851
std       37768.862451
min         730.000000
25%        2722.500000
50%        5756.000000
75%       12301.000000
max      217968.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,7319,217968,2293,2021-07-31,4058,3,20440.0,0.229697,0.254584,2.597518,13834925,157.549101,1575.491013,147.742037
1,大阪府,15201,114135,2721,2021-07-31,1040,0,5238.0,0.179138,0.258569,2.674912,8849635,128.971421,1289.714209,59.188882
2,神奈川県,7882,83348,987,2021-07-31,1580,1,7041.0,0.114245,0.174092,1.888483,9209442,90.502769,905.027688,76.454143
3,埼玉県,6193,56940,849,2021-07-31,1036,1,5113.0,0.214537,0.183378,2.002899,7390054,77.049505,770.495046,69.187586
4,愛知県,12949,54094,1008,2021-07-31,287,0,1384.0,0.247826,0.333327,3.159420,7575530,71.406225,714.062250,18.269349
5,千葉県,6756,49163,734,2021-07-31,792,0,3891.0,0.051793,0.189858,1.631229,6319772,77.792363,777.923634,61.568677
6,北海道,563,44424,1425,2021-07-31,284,1,1409.0,0.136000,0.152165,1.406780,5267762,84.331828,843.318282,26.747602
7,兵庫県,15764,43987,1316,2021-07-31,329,0,1600.0,0.241509,0.460220,3.633803,5549568,79.262025,792.620254,28.831073
8,福岡県,22520,39370,532,2021-07-31,504,0,2323.0,0.054393,0.292914,4.090909,5129841,76.747018,767.470181,45.284055
9,沖縄県,26461,24761,236,2021-07-31,439,0,2239.0,0.149215,0.421639,3.434343,1481547,167.129359,1671.293587,151.125816


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      46.329101
std       35.076315
min       10.350958
25%       24.074206
50%       31.774738
75%       56.480992
max      167.129359
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,26461,24761,236,2021-07-31,439,0,2239.0,0.149215,0.421639,3.434343,1481547,167.129359,1671.293587,151.125816
26,東京都,7319,217968,2293,2021-07-31,4058,3,20440.0,0.229697,0.254584,2.597518,13834925,157.549101,1575.491013,147.742037
32,神奈川県,7882,83348,987,2021-07-31,1580,1,7041.0,0.114245,0.174092,1.888483,9209442,90.502769,905.027688,76.454143
7,埼玉県,6193,56940,849,2021-07-31,1036,1,5113.0,0.214537,0.183378,2.002899,7390054,77.049505,770.495046,69.187586
5,千葉県,6756,49163,734,2021-07-31,792,0,3891.0,0.051793,0.189858,1.631229,6319772,77.792363,777.923634,61.568677
9,大阪府,15201,114135,2721,2021-07-31,1040,0,5238.0,0.179138,0.258569,2.674912,8849635,128.971421,1289.714209,59.188882
31,石川県,9571,5105,118,2021-07-31,90,0,635.0,-0.181818,0.211961,1.571429,1139612,44.795948,447.959481,55.720719
34,福岡県,22520,39370,532,2021-07-31,504,0,2323.0,0.054393,0.292914,4.090909,5129841,76.747018,767.470181,45.284055
1,京都府,14638,18331,249,2021-07-31,199,0,956.0,0.191617,0.362372,4.852941,2545899,72.002071,720.020708,37.550586
27,栃木県,5067,8324,85,2021-07-31,170,0,738.0,0.416667,0.322247,4.483871,1965516,42.350202,423.502022,37.547392


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    45.000000
mean      0.375805
std       0.274722
min       0.133294
25%       0.211961
50%       0.297197
75%       0.421639
max       1.656473
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
39,長崎県,23646,3461,69,2021-07-31,17,0,140.0,-0.564103,1.656473,16.000000,1350769,25.622442,256.224417,10.364466
43,香川県,20831,2272,32,2021-07-31,43,0,117.0,1.866667,0.906602,9.750000,981280,23.153432,231.534323,11.923202
11,宮城県,2252,9906,91,2021-07-31,65,0,255.0,0.300000,0.859435,12.000000,2292385,43.212637,432.126366,11.123786
13,富山県,9008,2286,38,2021-07-31,27,0,145.0,-0.250000,0.837075,8.000000,1055999,21.647748,216.477478,13.731074
33,福井県,10134,1606,36,2021-07-31,29,0,161.0,-0.033333,0.680371,4.800000,780053,20.588345,205.883446,20.639623
12,宮崎県,25335,3228,27,2021-07-31,14,0,73.0,-0.066667,0.677152,0.555556,1095903,29.455162,294.551616,6.661173
2,佐賀県,23083,2747,24,2021-07-31,23,0,111.0,-0.080000,0.632324,6.666667,823810,33.345067,333.450674,13.473981
36,秋田県,2815,1020,20,2021-07-31,7,0,30.0,0.000000,0.521429,2.500000,985416,10.350958,103.509584,3.044400
44,高知県,21957,2087,29,2021-07-31,14,0,98.0,-0.176471,0.477128,6.000000,709230,29.426279,294.262792,13.817802
3,兵庫県,15764,43987,1316,2021-07-31,329,0,1600.0,0.241509,0.460220,3.633803,5549568,79.262025,792.620254,28.831073


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
19,岩手県,1689,2002,47,2021-07-31,21,0,103.0,0.105263,0.133294,0.750000,1235517,16.203743,162.037430,8.336591
25,新潟県,8445,4017,54,2021-07-31,58,0,293.0,0.380952,0.143558,1.071429,2236042,17.964779,179.647788,13.103511
4,北海道,563,44424,1425,2021-07-31,284,1,1409.0,0.136000,0.152165,1.406780,5267762,84.331828,843.318282,26.747602
18,岡山県,18579,8185,127,2021-07-31,65,0,344.0,-0.155844,0.152598,1.096774,1903627,42.996869,429.968686,18.070767
20,島根県,18016,730,2,2021-07-31,17,0,68.0,0.214286,0.160147,1.428571,679324,10.745977,107.459769,10.009951
32,神奈川県,7882,83348,987,2021-07-31,1580,1,7041.0,0.114245,0.174092,1.888483,9209442,90.502769,905.027688,76.454143
7,埼玉県,6193,56940,849,2021-07-31,1036,1,5113.0,0.214537,0.183378,2.002899,7390054,77.049505,770.495046,69.187586
5,千葉県,6756,49163,734,2021-07-31,792,0,3891.0,0.051793,0.189858,1.631229,6319772,77.792363,777.923634,61.568677
21,広島県,19142,12141,179,2021-07-31,65,0,327.0,0.203704,0.191247,2.095238,2826858,42.948744,429.487438,11.567613
42,静岡県,12386,10885,157,2021-07-31,168,1,745.0,0.388430,0.198687,1.896552,3708556,29.351047,293.510466,20.088681


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  5  1  1  1  0  0  1]
[   730.    27884.75  55039.5   82194.25 109349.   136503.75 163658.5
 190813.25 217968.  ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([37.,  2.,  5.,  1.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 6., 20.,  9.,  7.,  2.,  0.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
19695,2021-07-22,35,山口県,4,3204,0,80,23.0,0.000000,NaN,3.000000
19696,2021-07-23,35,山口県,0,3204,0,80,19.0,-1.000000,NaN,-1.000000
19697,2021-07-24,35,山口県,2,3206,0,80,15.0,inf,NaN,-0.666667
19698,2021-07-25,35,山口県,5,3211,0,80,18.0,1.500000,NaN,1.500000
19699,2021-07-26,35,山口県,7,3218,0,80,25.0,0.400000,NaN,inf
19700,2021-07-27,35,山口県,9,3227,0,80,31.0,0.285714,NaN,2.000000
19701,2021-07-28,35,山口県,10,3237,0,80,37.0,0.111111,NaN,1.500000
19702,2021-07-29,35,山口県,16,3253,0,80,49.0,0.600000,NaN,3.000000
19703,2021-07-30,35,山口県,27,3280,0,80,76.0,0.687500,NaN,inf
19704,2021-07-31,35,山口県,17,3297,0,80,91.0,-0.370370,0.459136,7.500000
